In [1]:
import spacy
import explacy # simple parser visualization
import pandas as pd
import glob
from importlib import reload

In [2]:
nlp = spacy.load('en')

In [3]:
reload(explacy)

<module 'explacy' from '/root/w266_final/explacy.py'>

In [6]:
# suprisingly becomes child of was, but maybe this is more correct
# text = 'The salad was surprisingly tasty.'
# explacy.print_parse_info(nlp, text)

In [7]:
# this looks right... 
# text = 'The salad was unexpectedly tasty.'
# explacy.print_parse_info(nlp, text)

In [8]:
# bring 1 json into a dataframe for analysis
files = sorted(glob.glob('layer1.json.[0].valid.gz'))
df = pd.concat([pd.read_pickle(fp) for fp in files], ignore_index=True)
df.shape

(99999, 9)

In [9]:
# let's filter out epicurious recipes with less than 10 steps
keep_columns = ['id', 'partition', 
                'collection', 'title', 
                'ingr_count', 'instr_count', 
                'ingredients', 'instructions']

epicurious = df[(df.collection == 'www.epicurious.com') 
   & (df.instr_count<=5)][keep_columns].sort_values(by=['instr_count'], ascending=False)

# train, val, test counts
epicurious.groupby('partition').size()

partition
test     111
train    489
val      110
dtype: int64

In [60]:
# for simplicity, let's look at recipes with just less than 5 steps 
test_recipes = epicurious[(epicurious.instr_count<=5)].head(10).index

Int64Index([53425, 53356, 55816, 55722, 55493, 54858, 54715, 54065, 53479,
            53358],
           dtype='int64')


In [59]:
# Garlic Broth with Sage and Parsley, 21658
# Grilled Corn on the Cob, 96647

for recipe in test_recipes:
    # title
    print('\n' + epicurious['title'][recipe] + '\n')
    
    # ingredients
    for index, each in enumerate(epicurious['ingredients'][recipe]):
        print('Ingredient', index+1, ':', list(each.values())[0])
    
    # instructoins    
    for index, each in enumerate(epicurious['instructions'][recipe]):
        print('Step', index+1, ':', list(each.values())[0])


Confetti Salad with Ranch Dressing

Ingredient 1 : 1 English hothouse cucumber, chopped
Ingredient 2 : 1 16-ounce bag frozen petite peas, thawed
Ingredient 3 : 2 bunches radishes, chopped
Ingredient 4 : 1 cup chopped celery
Ingredient 5 : 1/2 cup finely chopped red onion
Ingredient 6 : Ranch Dressing
Step 1 : Combine all vegetables in large bowl.
Step 2 : (Can be prepared 4 hours ahead.
Step 3 : Cover and refrigerate.)
Step 4 : Toss salad with enough Ranch Dressing to coat.
Step 5 : Season to taste with salt and pepper and serve.

Mango Curry Truffles

Ingredient 1 : bittersweet chocolate truffle base
Ingredient 2 : 1 teaspoon mild curry powder
Ingredient 3 : 1/2 cup mango puree (from 1 mango)
Ingredient 4 : mango powder* or toasted finely shredded unsweetened coconut
Ingredient 5 : *A tart powder available at Indian markets and at savoryspiceshop.com
Step 1 : Follow recipe for bittersweet chocolate truffle base , stirring curry powder into hot cream (off heat); let stand 2 minutes.
S

In [68]:
# spaCy parameters under Token:
# Text: The original word text.
# Lemma: The base form of the word.
# POS: The simple part-of-speech tag.
# Tag: The detailed part-of-speech tag.
# Dep: Syntactic dependency, i.e. the relation between tokens.
# Shape: The word shape – capitalisation, punctuation, digits.
# is alpha: Is the token an alpha character?
# is stop: Is the token part of a stop list, i.e. the most common words of the language?

for recipe in test_recipes:
    # title
    print('\n' + epicurious['title'][recipe].upper() + '\n')
    
    # instructoins    
    for each in epicurious['instructions'][recipe]:
        instruction = list(each.values())[0]
        displacy.render(nlp(instruction), style='ent', jupyter=True)
        # print parser tree for each step
        explacy.print_parse_info(nlp, instruction)


CONFETTI SALAD WITH RANCH DRESSING



/root/anaconda3/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


Dep tree   Token      Dep   Tag Pos  
────────── ────────── ───── ─── ─────
┌┬──┬───── Combine    ROOT  VB  VERB 
││  │  ┌─► all        det   DT  DET  
││  └─►└── vegetables dobj  NNS NOUN 
│└─►┌───── in         prep  IN  ADP  
│   │  ┌─► large      amod  JJ  ADJ  
│   └─►└── bowl       pobj  NN  NOUN 
└────────► .          punct .   PUNCT


Dep tree   Token    Dep      Tag   Pos  
────────── ──────── ──────── ───── ─────
     ┌───► (        punct    -LRB- PUNCT
     │┌──► Can      aux      MD    VERB 
     ││┌─► be       auxpass  VB    VERB 
┌┬───┴┴┴── prepared ROOT     VBN   VERB 
││     ┌─► 4        nummod   CD    NUM  
││  ┌─►└── hours    npadvmod NNS   NOUN 
│└─►└───── ahead    advmod   RB    ADV  
└────────► .        punct    .     PUNCT


/root/anaconda3/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


...

Dep tree    Token   Dep      Tag Pos  
─────────── ─────── ──────── ─── ─────
       ┌──► Scoop   compound NNP PROPN
       │┌─► ice     compound NN  NOUN 
┌┬─────┴┴── cream   ROOT     NN  NOUN 
│└─►┌────── into    prep     IN  ADP  
│   │   ┌─► dessert compound NN  NOUN 
│   └─►┌┼── dishes  pobj     NNS NOUN 
│      │└─► or      cc       CC  CCONJ
│      └──► goblets conj     NNS NOUN 
└─────────► .       punct    .   PUNCT


/root/anaconda3/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


Dep tree Token Dep    Tag Pos  
──────── ───── ────── ─── ─────
┌┬┬───── Spoon ROOT   VB  VERB 
│││  ┌─► warm  amod   JJ  ADJ  
││└─►└── sauce dobj   NN  NOUN 
│└─────► over  advmod IN  ADP  
└──────► .     punct  .   PUNCT


/root/anaconda3/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


Dep tree   Token      Dep   Tag Pos  
────────── ────────── ───── ─── ─────
┌┬─────┬── Serve      ROOT  VB  VERB 
││     └─► ,          punct ,   PUNCT
│└─►┌───── passing    advcl VBG VERB 
│   │  ┌─► additional amod  JJ  ADJ  
│   └─►└── liqueur    dobj  NN  NOUN 
└────────► .          punct .   PUNCT


In [69]:
# for each in (epicurious['ingredients'].head(1)):
#     for k, v in enumerate(each):
#         doc = nlp(list(v.values())[0])
#         print(list(v.values())[0])
#         for chunk in doc.noun_chunks:
#             print(chunk.root)

In [ ]:
# import benepar
# benepar.download('benepar_en')

In [ ]:
# from benepar.spacy_plugin import BeneparComponent

In [ ]:
# nlp = spacy.load('en')
# nlp.add_pipe(BeneparComponent('benepar_en'))

In [ ]:
# doc = nlp("Bring some light chicken broth to a boil with a few leaves of fresh sage.")
# sent = list(doc.sents)[0]

In [ ]:
# for each in enumerate (sent._.children):
#     print(each)

In [ ]:
# type(sent._.children)

In [ ]:
# for each in enumerate (sent._.children):
#     # go from the start to the end of each span, returning each token in the sentence
#     # combine each token using join()
#     print(each.parse_string)